In [ ]:
import os
import pickle
import shutil
import json
from os.path import join as ojoin

In [ ]:
data_dir = os.path.abspath(ojoin('..','..','..','code','fairim','networks'))

In [ ]:
def get_attribute_values(network):
    data_fname = ojoin(data_dir, '%s.pickle'%network)
    with open(data_fname, 'rb') as f:
        data = pickle.load(f)    

    attribute_values = dict()
    for node_id, attributes in data.nodes(data=True):
        for k, v in attributes.items():
            if not k in attribute_values:
                attribute_values[k] = set()
            attribute_values[k].add(v)
            
    return attribute_values

In [ ]:
attribute_values = get_attribute_values('graph_spa_500_0')
for i in range(24):
    network = 'graph_spa_500_%d'%i
    av = get_attribute_values(network)
    assert av == attribute_values

for k, v in attribute_values.items():
    attribute_values[k] = list(v)

print(attribute_values)

In [ ]:
with open('attribute_values.json', 'w') as f:
    json.dump(attribute_values, f)

In [ ]:
def mkdir_fresh(dir_path):
    if os.path.isdir(dir_path):
        shutil.rmtree(dir_path)
    os.mkdir(dir_path)
    
# create one directory for each network
rootdir = os.path.abspath(ojoin('..', 'datasets'))
for network_id in range(24):
    network_name = 'graph_spa_500_%d'%network_id
    network_dir = ojoin(rootdir, network_name)
    mkdir_fresh(network_dir)

    # create one directory for each attribute
    for attribute in attribute_values:
        attribute_dir = ojoin(network_dir, attribute)
        mkdir_fresh(attribute_dir)
        
        # create one directory for each value
        for value in attribute_values[attribute]:
            value_dir = ojoin(attribute_dir, value)
            mkdir_fresh(value_dir)
            keep_file = ojoin(value_dir, '.keep')
            open(keep_file, 'a').close()